In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import imutils, sys, os, inspect
from keras.models import load_model
import multiprocessing, threading
import tensorflow as tf
sys.path.insert(0,"D:\\College\\2020\\GP\\Coding\\GP-Code") 

from data import DataSet


Using TensorFlow backend.
C:\Users\mahmo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mahmo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mahmo\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mahmo\AppData\Roa

In [2]:
import keras
import tensorflow as tf
keras.__version__, tf.__version__

('2.2.4', '1.14.0')

In [3]:
sequence_list = []

#capturing live video frames
model_frames_num = 30
realtime_frames = 120
saved_model = 'D:/College/2020/GP/Coding/GP-Code/data/checkpoints/lrcn-images.385-0.111.hdf5'
model = load_model(saved_model, compile=False)
graph = tf.get_default_graph()
model.summary()

W0906 16:12:05.542775 16500 deprecation_wrapper.py:119] From C:\Users\mahmo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0906 16:12:05.564682 16500 deprecation_wrapper.py:119] From C:\Users\mahmo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0906 16:12:05.613576 16500 deprecation_wrapper.py:119] From C:\Users\mahmo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:245: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0906 16:12:05.615546 16500 deprecation_wrapper.py:119] From C:\Users\mahmo\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0906 16:12:05.617543 16500 deprecation_wrapper.py:119]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 150, 150, 32)  4736      
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 150, 150, 32)  128       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 150, 150, 32)  0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 148, 148, 32)  9248      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 148, 148, 32)  128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 148, 148, 32)  0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 74, 74, 32)    0         
__________

In [4]:
data = DataSet(seq_length=model_frames_num, class_limit=None)

Max frames => 300


In [5]:
def predictions(frames, model):
    assert len(frames) == model_frames_num
    with graph.as_default():
        prediction = model.predict(np.expand_dims(frames, axis=0))
        pred = data.print_class_from_prediction(np.squeeze(prediction, axis=0))
        print(pred)
        return pred[0][0]

In [6]:
# Number of frames to pass before changing the frame to compare the current
# frame against
FRAMES_TO_PERSIST = 10

# Minimum boxed area for a detected motion to count as actual motion
# Use to filter out noise or small objects
MIN_SIZE_FOR_MOVEMENT = 2000

# Minimum length of time where no motion is detected it should take
#(in program cycles) for the program to declare that there is no movement
MOVEMENT_DETECTED_PERSISTENCE = 100

In [9]:
# Create capture object
cap = cv2.VideoCapture(5) # Flush the stream
cap.release()
cap = cv2.VideoCapture("1.mp4") # Then start the webcam
# Deployment\Vid\alaykom_2.mp4
# Init frame variables
first_frame = None
next_frame = None

# Init display font and timeout counters
font = cv2.FONT_HERSHEY_SIMPLEX
delay_counter = 0
movement_persistent_counter = 0

dim = (300, 300)
p = ''
wait = 0
process_list = []

In [10]:
while True:
    
    wait += 1 
    # Set transient motion detected as false
    transient_movement_flag = False
    
    # Read frame
    ret, frame = cap.read()
    text = "Unoccupied"

    # If there's an error in capturing
    if not ret:
        print("CAPTURE ERROR")
        break

#     frame = cv2.flip(frame, 1)
#     full_fram = frame.copy()
#     full_fram = cv2.resize(frame, (800,600), interpolation = cv2.INTER_AREA)
#     frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    
#     sequence_list.append(frame)
        
#     if len(sequence_list) == realtime_frames:
#         rescaledList = data.rescale_list(sequence_list, model_frames_num)
#         p = predictions(rescaledList)
#         sequence_list = []
#         rescaledList = []
        
#     cv2.putText(full_fram, p, (250,550), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_AA)
    
    # Resize and save a greyscale version of the image    
    frame = imutils.resize(frame, width = 750)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Blur it to remove camera noise (reducing false positives)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

    # If the first frame is nothing, initialise it
    if first_frame is None: first_frame = gray    

    delay_counter += 1

    # Otherwise, set the first frame to compare as the previous frame
    # But only if the counter reaches the appriopriate value
    # The delay is to allow relatively slow motions to be counted as large
    # motions if they're spread out far enough
    if delay_counter > FRAMES_TO_PERSIST:
        delay_counter = 0
        first_frame = next_frame

        
    # Set the next frame to compare (the current frame)
    next_frame = gray

    # Compare the two frames, find the difference
    frame_delta = cv2.absdiff(first_frame, next_frame)
    thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)[1]

    # Fill in holes via dilate(), and find contours of the thesholds
    thresh = cv2.dilate(thresh, None, iterations = 2)
    _, cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # loop over the contours
    for c in cnts:

        # Save the coordinates of all found contours
        (x, y, w, h) = cv2.boundingRect(c)
        
        # If the contour is too small, ignore it, otherwise, there's transient
        # movement
        if cv2.contourArea(c) > MIN_SIZE_FOR_MOVEMENT:
            transient_movement_flag = True
            
            # Draw a rectangle around big enough movements
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # The moment something moves momentarily, reset the persistent
    # movement timer.
    if transient_movement_flag == True:
        movement_persistent_flag = True
        movement_persistent_counter = MOVEMENT_DETECTED_PERSISTENCE

    # As long as there was a recent transient movement, say a movement
    # was detected    
    if movement_persistent_counter > 0:
        text = "Movement Detected " + str(movement_persistent_counter)
        movement_persistent_counter -= 1
    else:
        text = "No Movement Detected"

# q = multiprocessing.JoinableQueue()

    if wait > 30:
        if movement_persistent_counter > 85 :
            pred_frame = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
            sequence_list.append(pred_frame)
        else:
            if len(sequence_list) > 30:
                rescaledList = data.rescale_list(sequence_list, model_frames_num)
                thread = threading.Thread(target = predictions, args=(rescaledList, model, ))
                thread.daemon = True
                thread.start()
#                 p = predictions(rescaledList)
                sequence_list = []
                rescaledList = []

        
    # Print the text on the screen, and display the raw and processed video 
    # feeds
    cv2.putText(frame, str(text), (10,35), font, 0.75, (255,255,255), 2, cv2.LINE_AA)
    
    # For if you want to show the individual video frames
#    cv2.imshow("frame", frame)
#    cv2.imshow("delta", frame_delta)
    
    # Convert the frame_delta to color for splicing
    frame_delta = cv2.cvtColor(frame_delta, cv2.COLOR_GRAY2BGR)

    # Splice the two video frames together to make one long horizontal one
    cv2.imshow("frame", np.hstack((frame_delta, frame)))

    # Interrupt trigger by pressing q to quit the open CV program
    ch = cv2.waitKey(1)
    if ch & 0xFF == ord('q'):
        break
        
# Cleanup when closed
# cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

[('saydalya', 0.83472574), ('alhamd', 0.043138072), ('masged', 0.037743494), ('tamam', 0.020178216), ('salam', 0.013875334)]
[('saydalya', 0.82027614), ('masged', 0.04436619), ('amel', 0.030610064), ('alhamd', 0.0239482), ('salam', 0.0232721)]
[('saydalya', 0.8530785), ('masged', 0.027147565), ('salam', 0.02323856), ('alhamd', 0.020455156), ('tamam', 0.012516883)]
[('saydalya', 0.8563655), ('salam', 0.031138064), ('alhamd', 0.027657626), ('masged', 0.022336852), ('amel', 0.012832711)]
[('saydalya', 0.6811314), ('salam', 0.06528317), ('alhamd', 0.05143362), ('amel', 0.044286605), ('masged', 0.04234745)]
[('saydalya', 0.7311544), ('masged', 0.06087138), ('alhamd', 0.047942422), ('salam', 0.039781213), ('amel', 0.03126263)]
[('saydalya', 0.7306542), ('alhamd', 0.07474667), ('amel', 0.052180756), ('masged', 0.03161785), ('salam', 0.024760325)]
[('saydalya', 0.8304806), ('masged', 0.037607852), ('alhamd', 0.029145839), ('ana', 0.022861145), ('salam', 0.018191751)]
[('saydalya', 0.7659167), 

In [9]:
# import threading
# import time


# class ThreadingExample(object):
#     """ Threading example class
#     The run() method will be started and it will run in the background
#     until the application exits.
#     """

#     def __init__(self, interval=1):
#         """ Constructor
#         :type interval: int
#         :param interval: Check interval, in seconds
#         """
#         self.interval = interval

#         thread = threading.Thread(target=self.run, args=())
#         thread.daemon = True                            # Daemonize thread
#         thread.start()                                  # Start the execution

#     def run(self):
#         """ Method that runs forever """
#         while True:
#             # Do something
#             print('Doing something imporant in the background')

#             time.sleep(self.interval)

# example = ThreadingExample()
# time.sleep(3)
# print('Checkpoint')
# time.sleep(2)
# print('Bye')